# <center>DLRM model</center>

### Import depencies

In [1]:
from IPython.display import clear_output

In [2]:
!python -c "import fbgemm_gpu"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/royzalta/anaconda3/lib/python3.11/site-packages/fbgemm_gpu/__init__.py", line 12, in <module>
    torch.ops.load_library(os.path.join(os.path.dirname(__file__), "fbgemm_gpu_py.so"))
  File "/Users/royzalta/anaconda3/lib/python3.11/site-packages/torch/_ops.py", line 643, in load_library
    ctypes.CDLL(path)
  File "/Users/royzalta/anaconda3/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen(/Users/royzalta/anaconda3/lib/python3.11/site-packages/fbgemm_gpu/fbgemm_gpu_py.so, 0x0006): tried: '/Users/royzalta/anaconda3/lib/python3.11/site-packages/fbgemm_gpu/fbgemm_gpu_py.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/royzalta/anaconda3/lib/python3.11/site-packages/fbgemm_gpu/fbgemm_gpu_py.so' (no such file), '/Users/royzalta/anaconda3/lib/python3.11/site-packages/fbgem

In [3]:
# Release GPU
conda install pytorch cuda -c pytorch -c "nvidia/label/cuda-11.7.1"
!pip install fbgemm-gpu

SyntaxError: invalid syntax (342062524.py, line 2)

In [ ]:
!pip install fbgemm-gpu
#!pip install fbgemm-gpu-cpu

!pip install torch
!pip install torchrec
#!pip install torchrec-cpu
!pip install pyre_extensions

#clear_output()

In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools

In [ ]:
conda update conda

In [ ]:
import pandas as pd
import numpy as np
import random

import torch
from torchrec.datasets.utils import Batch
from torchrec.sparse.jagged_tensor import KeyedJaggedTensor
from sklearn.model_selection import train_test_split
#import custom function from book_funcs file
from book_funcs import category_cleaner, df_cleaner, category_cleaner,category_compliter, zero_droper, precentage_null

In [ ]:
from batch import *
from model import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

### Import Data

In [ ]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

In [ ]:
df['age'] = df['age'].astype(int)
df['year_of_publication'] = df['year_of_publication'].astype(int)

### train & test split

In [ ]:
train_df, test_df = train_test_split(
    df, test_size=0.33, random_state=42)

### Apply custom transformers

In [ ]:
train_df = category_cleaner(train_df)
train_df = df_cleaner(train_df)
train_df = category_compliter(train_df)
train_df = zero_droper(train_df)

### Handle null values

In [ ]:
null_train = precentage_null(train_df)
print(null_train)

In [ ]:
columns_to_check = ["city", "state", "country"]
train_df = train_df.dropna(subset=columns_to_check, how="any")

In [ ]:
null_test = precentage_null(test_df)
print(null_test)

In [ ]:
columns_to_check = ["city", "state", "country"]
test_df = test_df.dropna(subset=columns_to_check, how="any")

### Encoding sparse features

In [ ]:
cols_dense = ["age", "year_of_publication"]
cols_sparse = ['location',
               'publisher','Language','Category',
              'city','state','country']

In [ ]:
def encode(col, map_rev):
    encoded_values = []
    for item in col:
        if pd.notna(item):  # Check if the value is not NaN
            encoded_value = map_rev.get(item, item)
            encoded_values.append(encoded_value)
        else:
            # Handle NaN values as needed
            encoded_values.append('NaN')
    return encoded_values


In [ ]:
map_sparse = {}
map_sparse_rev = {}

In [ ]:
for feat in cols_sparse:
    map_sparse[feat] = {i: c for i,c in enumerate(train_df[feat].explode().value_counts().index)}
    map_sparse_rev[feat] = {v:k for k, v in map_sparse[feat].items()}

    train_df[feat+'_enc'] = train_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))
    test_df[feat+'_enc'] = test_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))


In [ ]:
map_sparse

### Build data batches

In [ ]:
train_data = RecBatch(
    data = train_df,
    cols_sparse = [c+'_enc' for c in cols_sparse],
    cols_dense = cols_dense,
    col_label = "label",
    batch_size = 100,
    num_generated_batches = 20,
    seed = 123,
    device = device
)